# Cloning and Updating CLSS Dashboard References in ArcGIS Online

This notebook provides a process to clone the "Dev-CLSS-Dashboard" from one ArcGIS Online organization to another. During the cloning process, all references to the original CLSS Feature Service are updated to point to a new target feature service. This workflow is useful for migrating dashboards between environments or organizations while ensuring data source references remain accurate.

In [ ]:
from arcgis.gis import GIS
import json
import re
import os
from dotenv import load_dotenv


### --- SETUP: First configure key parameters and establish connection to source/target Orgs

In [ ]:
# configure an .env file with the following variables:
# SRC_USERNAME, SRC_PASSWORD, TRG_USERNAME, TRG_PASSWORD
load_dotenv()
SRC_USERNAME = os.getenv("SRC_USERNAME")
SRC_PASSWORD = os.getenv("SRC_PASSWORD")
TRG_USERNAME = os.getenv("TRG_USERNAME")
TRG_PASSWORD = os.getenv("TRG_PASSWORD")

# --- CONFIGURATION ---
source_org = GIS("https://ghis.maps.arcgis.com", SRC_USERNAME, SRC_PASSWORD)
target_org = GIS("https://cusec-hub.maps.arcgis.com", TRG_USERNAME, TRG_PASSWORD)

# IDs and URLs
source_dashboard_id = "6d275452758949099bc70fd9718a9862"
source_feature_id = "649091ebf4af4233902e975798d59ef9"
target_feature_id = "ee81054b84fa436bb04860d36845d1fd"

# Optional: use URLs if referenced by URL
source_feature_url = f"https://services3.arcgis.com/j2a3SeWN04oskFYa/arcgis/rest/services/CLSS_FeatureService_Demo/FeatureServer"
target_feature_url = f"https://services3.arcgis.com/j2a3SeWN04oskFYa/arcgis/rest/services/CLSS_FeatureService_Dev/FeatureServer"


### --- STEP 1: Fetch the source dashboard ---

In [12]:
source_dashboard = source_org.content.get(source_dashboard_id)
dashboard_data = source_dashboard.get_data()
dashboard_text = dashboard_data.decode('utf-8') if isinstance(dashboard_data, bytes) else json.dumps(dashboard_data)

### --- STEP 2: Replace feature service references ---

In [ ]:
# Replace by Item ID (used internally for dataSources, layer references, etc.)
dashboard_text = re.sub(source_feature_id, target_feature_id, dashboard_text)

# Replace by URL (used in some layout or config blocks)
dashboard_text = dashboard_text.replace(source_feature_url, target_feature_url)

print(f"References to feature service replaced.")

References to feature service replaced.


### --- STEP 3: Create the dashboard in the target org ---

In [ ]:
# Copy item properties
item_properties = {
    'title': source_dashboard.title + " (Cloned)",
    'type': 'Dashboard',
    'tags': source_dashboard.tags,
    'snippet': source_dashboard.snippet,
    'text': dashboard_text 
}

# Add the cloned dashboard
cloned_dashboard = target_org.content.add(item_properties)
print(f"Dashboard cloned to target org: {cloned_dashboard.title} (ID: {cloned_dashboard.id})")

# Share as needed
# cloned_dashboard.share(org=True)  # or share(everyone=True)


c:\Code\clss\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Dashboard cloned to target org: Dev-CLSS-Dashboard (Cloned) (ID: 90d46a8565d94828b5a4a26e61ff9f8c)
